# Evalutaion
To be able to make a statement about the performance of a question-asnwering system, it is important to evalute it. Furthermore, evaluation allows to determine which parts of the system can be improved.

## Start an Elasticsearch server
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (eg., in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [1]:
# Recommended: Start Elasticsearch using Docker
! docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.6.2

a844e3ec4f41b5d2b24fe3d562e8302896baea1d0a761295998434c2de490714


In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
#! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
#! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
#! chown -R daemon:daemon elasticsearch-7.6.2

#import os
#from subprocess import Popen, PIPE, STDOUT
#es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
#! sleep 30

In [2]:
from farm.utils import initialize_device_settings

device, n_gpu = initialize_device_settings(use_cuda=True)

05/19/2020 09:03:25 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None


In [3]:
# Connect to Elasticsearch
from haystack.database.elasticsearch import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", create_index=False)

In [4]:
# Add evaluation data to Elasticsearch database
document_store.add_eval_data("../data/natural_questions/dev_subset.json")

05/19/2020 09:03:37 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.796s]
05/19/2020 09:03:38 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.222s]


## Initialize components of QA-System

In [5]:
# Initialize Retriever
from haystack.retriever.elasticsearch import ElasticsearchRetriever

retriever = ElasticsearchRetriever(document_store=document_store)

In [6]:
# Initialize Reader
from haystack.reader.farm import FARMReader

reader = FARMReader("deepset/roberta-base-squad2")

05/19/2020 09:03:46 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
05/19/2020 09:03:46 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
05/19/2020 09:03:50 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
05/19/2020 09:03:56 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
05/19/2020 09:04:02 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
05/19/2020 09:04:02 - INFO - farm.infer -   Got ya 7 parallel workers to do inference ...
05/19/2020 09:04:02 - INFO - farm.infer -    0    0    0    0  

In [7]:
# Initialize Finder which sticks together Reader and Retriever
from haystack.finder import Finder

finder = Finder(reader, retriever)

## Evaluation of Retriever

In [8]:
# Evaluate Retriever on its own
retriever_eval_results = retriever.eval()

## Retriever Recall is the proportion of questions for which the correct document containing the answer is
## among the correct documents
print("Retriever Recall:", retriever_eval_results["recall"])
## Retriever Mean Avg Precision rewards retrievers that give relevant documents a higher rank
print("Retriever Mean Avg Precision:", retriever_eval_results["mean avg precision"])

05/19/2020 09:04:11 - INFO - elasticsearch -   GET http://localhost:9200/feedback/_search?scroll=5m&size=1000 [status:200 request:0.090s]
05/19/2020 09:04:11 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.051s]
05/19/2020 09:04:11 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:11 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.013s]
05/19/2020 09:04:11 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:11 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.012s]
05/19/2020 09:04:11 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:11 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.012s]
05/19/2020 09:04:11 - INFO - haystack.retriever.elasticsearch -   Got 10 

05/19/2020 09:04:12 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:12 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.008s]
05/19/2020 09:04:12 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:12 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.008s]
05/19/2020 09:04:12 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:12 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.008s]
05/19/2020 09:04:12 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:12 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.006s]
05/19/2020 09:04:12 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09

Retriever Recall: 1.0
Retriever Mean Avg Precision: 0.984934086629002


## Evaluation of Reader

In [9]:
# Evaluate Reader on its own
reader_eval_results = reader.eval(document_store=document_store, device=device)

# Evaluation of Reader can also be done directly on a SQuAD-formatted file 
# without passing the data to Elasticsearch
#reader_eval_results = reader.eval_on_file("../data/natural_questions", "dev_subset.json", device=device)

## Reader Top-N-Recall is the proportion of predicted answers that overlap with their corresponding correct answer
print("Reader Top-N-Recall:", reader_eval_results["top_n_recall"])
## Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
## Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

05/19/2020 09:04:22 - INFO - elasticsearch -   GET http://localhost:9200/feedback/_search?scroll=5m&size=1000 [status:200 request:0.007s]
05/19/2020 09:04:22 - INFO - elasticsearch -   GET http://localhost:9200/_search/scroll [status:200 request:0.003s]
05/19/2020 09:04:22 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.001s]
05/19/2020 09:04:22 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search?scroll=5m&size=1000 [status:200 request:0.014s]
05/19/2020 09:04:22 - INFO - elasticsearch -   GET http://localhost:9200/_search/scroll [status:200 request:0.002s]
05/19/2020 09:04:22 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.002s]
Evaluating: 100%|██████████| 64/64 [00:14<00:00,  4.28it/s]

Reader Top-N-Recall: 0.5084745762711864
Reader Exact Match: 0.23728813559322035
Reader F1-Score: 0.23728813559322035


## Evaluation of Finder

In [10]:
# Evaluate combination of Reader and Retriever through Finder
finder_eval_results = finder.eval()

print("Retriever Recall in Finder:", finder_eval_results["retriever_recall"])
print("Retriever Mean Avg Precision in Finder:", finder_eval_results["retriever_map"])

# Reader is only evaluated with those questions, where the correct document is among the retrieved ones
print("Reader Recall in Finder:", finder_eval_results["reader_recall"])
print("Reader Mean Avg Precision in Finder:", finder_eval_results["reader_map"])
print("Reader Exact Match in Finder:", finder_eval_results["reader_em"])
print("Reader F1-Score in Finder:", finder_eval_results["reader_f1"])

05/19/2020 09:04:57 - INFO - elasticsearch -   GET http://localhost:9200/feedback/_search?scroll=5m&size=1000 [status:200 request:0.006s]
05/19/2020 09:04:57 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.006s]
05/19/2020 09:04:57 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:57 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.007s]
05/19/2020 09:04:57 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:57 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.005s]
05/19/2020 09:04:57 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:57 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.006s]
05/19/2020 09:04:57 - INFO - haystack.retriever.elasticsearch -   Got 10 

05/19/2020 09:04:58 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:58 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.005s]
05/19/2020 09:04:58 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:58 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.005s]
05/19/2020 09:04:58 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:58 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.004s]
05/19/2020 09:04:58 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09:04:58 - INFO - elasticsearch -   GET http://localhost:9200/eval_document/_search [status:200 request:0.006s]
05/19/2020 09:04:58 - INFO - haystack.retriever.elasticsearch -   Got 10 candidates from retriever
05/19/2020 09

Inferencing Samples: 100%|██████████| 3/3 [00:00<00:00,  5.49 Batches/s]


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  8.48 Batches/s]


Inferencing Samples: 100%|██████████| 2/2 [00:00<00:00,  4.00 Batches/s]


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 12.93 Batches/s]


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.27 Batches/s]
05/19/2020 09:09:54 - INFO - haystack.finder -   57 out of 59 questions were correctly answered (96.61%).
05/19/2020 09:09:54 - INFO - haystack.finder -   0 questions could not be answered due to the retriever.
05/19/2020 09:09:54 - INFO - haystack.finder -   2 questions could not be answered due to the reader.


Retriever Recall in Finder: 1.0
Retriever Mean Avg Precision in Finder: 0.984934086629002
Reader Recall in Finder: 0.9661016949152542
Reader Mean Avg Precision in Finder: 0.44187516814635447
Reader Exact Match in Finder: 0.9661016949152542
Reader F1-Score in Finder: 0.9661016949152542
